In [107]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))
import pandas as pd
import numpy as np
import spacy
import make_dataset as mk
import pickle
import csv
from openpyxl import load_workbook
import shutil 
import pycountry
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim

'''helper functions'''
import make_dataset as mk # make dataset - data processing

import make_dataset_PIMS_API as mkp 

import visualize as vis # visualisations 

import clean_dataset as clean # data cleaning

In [108]:
'''import data'''

landscape = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/landscape.csv')

technical_team = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/technical_team.csv')

strategies = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/strategies.csv')
pathways = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/pathways.csv')
targeted_risks = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/targeted_risks.csv')
transformed_sectors = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/transformed_sectors.csv')
undp_roles = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/undp_roles.csv')
_conventions_protocols_plans = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/_conventions_protocols_plans.csv')
_gender_equality = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/_gender_equality.csv')
_hot_topics = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/_hot_topics.csv')
_social_inclusion___engagement = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/_social_inclusion___engagement.csv')
_types_of_private_sector = pd.read_csv(os.path.abspath(os.path.join('..', 'data/processed'))+'/_types_of_private_sector.csv')

'''logframes'''
df_logs = pd.read_excel(os.path.abspath(os.path.join('..', 'data/interim'))+'/pims_logframes.xlsx')  


'''uncleaned data'''
taxonomy_filtered = pd.read_excel(os.path.abspath(os.path.join('..', 'data/interim'))+'/taxonomy_filtered.xlsx') 
all_text = taxonomy_filtered[['PIMS_ID', 'all_text']]

In [109]:
'''merge to master data_table'''
df = landscape.merge(technical_team, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(strategies, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(pathways, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(targeted_risks, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(transformed_sectors, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(undp_roles, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(_conventions_protocols_plans, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(_gender_equality, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(_hot_topics, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(_social_inclusion___engagement, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])
df = df.merge(_types_of_private_sector, how='left', on=['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy'])

df = df.merge(all_text, how='left', on=['PIMS_ID'])

df = df.drop(columns=['Unnamed: 0_x',
                      'Unnamed: 0_y', 'Unnamed: 0', 'Unnamed: 4_x', 
                      'Unnamed: 4_y','no tag',  'no tag_x', 'no tag_y'])


df = df.fillna(0)

In [110]:

df_org = df[['PIMS_ID', 'all_text', 'all_text_clean', 'all_text_clean_spacy']]
df_num = df._get_numeric_data()
df_num = df_num.drop(columns=['PIMS_ID'])
df = pd.concat([df_org, df_num], axis=1)

## import raw data again for merging purpose

In [111]:
wb = load_workbook(filename=(os.path.abspath(os.path.join('..', 'data/raw'))+'/urls.xlsx'))
ws = wb.active
# iterate thru all cells and if hyperlink found attempt modification of cell
for row in ws.rows:
    for cell in row:
        try:
            if len(cell.hyperlink.target)  > 0:
                cell.value = "".join([cell.value,"|",cell.hyperlink.target])
                # Join cell.value and hyperlink target into string (optionally just assign the hyperlink.target to the cell.value
        except:
            pass
        
# save workbook to temp .xlsx (I could not manage to read from buffer...) .
wb.save((os.path.abspath(os.path.join('..', 'data/interim'))+'/temp.xlsx'))

# read with pandas 
data = pd.read_excel((os.path.abspath(os.path.join('..', 'data/interim'))+'/temp.xlsx'))

# take DataSeries and rsplit by "|" and expand to 2 columns
hyper = (data.ProDoc_link.str.rsplit("|", expand=True))

#set labels
hyper.columns=["Label","hyperlink"]

# join them back to dataframe on index
data = data.join(hyper, how="left")

In [112]:
data = data.rename(columns={"PIMS #": "PIMS_ID", "TITLE": "title", 
                            "LEADING COUNTRY": "leading_country", 
                            "Grant amount": "grant_amount"})
data = data[['PIMS_ID', 'hyperlink', 'title', "leading_country","grant_amount" ]]

In [113]:
# merge with hyperlink:
df = df.merge(data, how='left', on=['PIMS_ID'])
df[['hyperlink']] = df[['hyperlink']].fillna(value="")

In [114]:
df['leading_country'] = df['leading_country'].str.replace("New York - GEF","USA")
df['leading_country'] = df['leading_country'].str.replace("Bangkok Regional Hub","Thailand")
df['leading_country'] = df['leading_country'].str.replace("Regional Centre - Istanbul","Turkey")


input_countries = df.leading_country.tolist()
countries = {}
for index, row in df.iterrows():
    for country in pycountry.countries:
        
        countries[country.name] = country.alpha_2

codes = [countries.get(country, 'Unknown code') for country in input_countries]
df = df.assign(country_code=codes)

In [116]:
to_iterate = df[['PIMS_ID', 'leading_country']]
geolocator = Nominatim(user_agent = "ich")
location= list()
for i in range(len(to_iterate)):
    location.append(geolocator.geocode(to_iterate.iloc[i, 1], timeout = 1))
    
location = [geolocator.geocode("Germany") if v is None else v for v in location]

In [117]:
l = [i for i in location if i is not None]
names = [l[i][0] for i in range(len(l))]
y = [l[i][1][0] for i in range(len(l))]
x = [l[i][1][1] for i in range(len(l))]
xy = [l[i][1] for i in range(len(l))]
df = df.assign(lon=x)
df = df.assign(lat=y)

In [118]:
"""save final dataset"""
df.to_csv(os.path.abspath(os.path.join('..', 'data/processed/'))+"/taxonomy_final.csv", index=False)

In [119]:
# """landscape"""

# parents = ['forests','conserve_areas', 'freshwaters', 'grasslands', 'desert',
#        'marine', 'wetlands', 'human_altered_areas']


# forests = ['tropical_forests', 'temperate_forests', 'dryland_forests',
#        'montane_forests', 'intact_forests', 'boreal_forests_taiga_forests']

# conserve_areas = ['marine_and_coastal_protected_areas',
#        'terrestrial_protected_areas',
#        'indigenous_and_communities_conserved_areas_iccas',
#        'transboundary_conservation_areas', 'productive_landscapes_seascapes',
#        'key_biodiversity_areas_kbas',
#        '_important_bird_and_biodiversity_areas_ibas',
#        'specially_protected_areas_spas', 'protected_areas_network',
#        'oecm_other_effective_area_based_conservation_measures',
#        'locally_managed_marine_areas']

# freshwaters = [ 'rivers_and_river_basins', 'lakes', 'aquifers',
#        'estuaries']

# grasslands = ['grazing_lands', 'tropical_grasslands',
#        'temperate_grasslands', 'savannas', 'steppes', 'drylands']

# desert = ['sub_tropical_hot_and_dry_deserts', 'rift_valley',
#        'semi_arid_cold_winter_deserts']

# marine = ['seas', 'coasts', 'coral_reefs', 'seagrasses',
#        'large_marine_ecosystem', 'exclusive_economic_zone',
#        'areas_beyond_national_jurisdiction']

# wetlands = ['mangroves', 'marshes', 'swamps', 'peatlands']

# human_altered_areas = ['urban_areas', 'rural_areas', 'mining_sites',
#        'industrial_site',
#        'contaminated_sites']

# sub_categories = [forests, conserve_areas, freshwaters, grasslands, desert, marine,
#                  wetlands, human_altered_areas]


# d_1 = dict(zip(parents, sub_categories)) 

'''import dict'''
with open(os.path.abspath(os.path.join('..', 'data/processed/'))+"/landscape_dict.pkl", 'rb') as handle:
    landscape_dict = pickle.load(handle)

In [120]:
# """strategies"""

# parents = ['capacity_building', 'enabling', 'finance_economy',
#        'food_and_agricultural_commodities', 'governance', 'law_regulation',
#        'management_operation', 'mitigation_adaptation', 'monitor_inventory',
#        'technology_innovation']

# capacity_building = ['awareness_raising', 'research',
#        'institutional_capacity_building', 'community_capacity_building',
#        'technical_capacity_building', 'education_courses',
#        'traditional_knowledge_application', 'portfolio_learning']

# enabling = ['mainstream', 'alignment', 'advocacy_towards_policy_makers',
#        'public_campaign', 'community_engagement']

# facs = ['improved_diversified_sustainable_livelihoods_tp_5',
#        'sustainable_agriculture_practices_and_use_of_resources_tp_2_7_8_9',
#        'climate_resilience_for_fac_system_tp_6',
#        'facs_strategies_for_sustainable_supply_chain_tp_4',
#        'crops_diversity_nutrition_and_abs_tp_1_10']

# finance_economy = ['nature_finance', 'energy_finance',
#        'circular_economy', 'blue_economy', 'green_economy', 'fiscal_planning',
#        'new_other_financial_schemes_mechanism']

# governance = ['_cooperative', 'institutional_framework', 'partnerships',
#        'transboundary_governance', 'inter_sectoral_coordination',
#        'adaptive_governance', 'participatory_governance_models',
#        'development_planning']

# law_regulation = ['laws_policy_plan_formulation',
#        'standards_labeling_guideline', 'laws_enforcement_regulation',
#        'pollution_control', 'surveillance_compliance',
#        'land_rights_and_tenure_security', 'conflict_resolution']

# management_operation = ['mainstream', 'alignment', 'advocacy_towards_policy_makers',
#        'public_campaign', 'community_engagement']

# mitigation_adaptation =['species_adaptation',
#        'ecosystem_mitigation_and_adaptation',
#        'drought_mitigation_early_warning', 'flood_prevention_early_warning',
#        'sustainable_fire_management', 'erosion_prevention',
#        'coastal_risk_reduction', 'sea_level_rise',
#        'infrastructure_against_natural_hazards', 'storm_mitigation',
#        'blue_carbon']

# monitor_inventory = ['data_quality', 'impact_assessment',
#        'waste_pollutants_monitoring', 'water_quality_quantity',
#        'ecological_monitoring', 'spatial_monitoring_analysis',
#        'ecosystem_services_monitoring', 'management_effectiveness_mett',
#        'accounting', 'knowledge_data_management']

# technology_innovation = ['mainstream', 'alignment', 'advocacy_towards_policy_makers',
#        'public_campaign', 'community_engagement']

# sub_categories = [capacity_building, enabling, facs, finance_economy, governance, law_regulation,
#                  management_operation, mitigation_adaptation, monitor_inventory, technology_innovation]


# d_2 = dict(zip(parents, sub_categories))

'''import dict'''
with open(os.path.abspath(os.path.join('..', 'data/processed/'))+"/strategy_dict.pkl", 'rb') as handle:
    strategy_dict = pickle.load(handle)

In [121]:
parents = ['capacity_building', 'enabling', 'finance_economy',
       'food_and_agricultural_commodities', 'governance', 'law_regulation',
       'management_operation', 'mitigation_adaptation', 'monitor_inventory',
       'technology_innovation']

strategy_categories = []
for i in strategy_dict.items():
    strategy_categories.append(i)
strategy_dict

{'capacity_building': ['awareness_raising',
  'technical_capacity_building',
  'institutional_capacity_building',
  'community_capacity_building',
  'mainstream',
  'alternative_energy_sources',
  'green_building_practices',
  'improved_soil_and_water_management_techniques',
  'impact_assessment',
  'alignment',
  'institutional_framework',
  '_institutional_capacity_building',
  'education_courses',
  'laws_policy_plan_formulation',
  'research',
  'innovations_in_techniques_approaches',
  'traditional_knowledge_application',
  'improved_diversified_sustainable_livelihoods_tp_5',
  'demonstration_sites_pilot',
  'energy_finance',
  'portfolio_learning',
  'partnerships',
  'fiscal_planning',
  'laws_enforcement_regulation',
  'knowledge_data_management',
  'community_engagement',
  'public_campaign',
  'advocacy_towards_policy_makers',
  'new_other_financial_schemes_mechanism',
  'facs_strategies_for_sustainable_supply_chain_tp_4',
  'sustainable_agriculture_practices_and_use_of_resou

In [122]:
# """technical team"""

# parents = ['chemicals_and_waste_programme', 'climate_change_adaptation_programme',
#        'climate_forest_of_climate_change_mitigation',
#        'climate_strategies_and_policy_programme',
#        'ecosystems_and_biodiversity_programme',
#        'energy_program_of_climate_change_mitigation',
#        'oceans_and_water_programme', 'environmental_governance_programme']

# chemicals_and_waste_programme = ['persistent_organic_pollutants',
#        'ozone_depleting_substances', 'heavy_metals', 'waste_management',
#        'cooling_alternatives']

# climate_change_adaptation_programme = ['climate_information_and_early_warning_systems_ci_ews',
#        'cross_sectoral_climate_resilient_livelihoods',
#        'fostering_food_security_and_resilient_agricultural_systems',
#        'mainstreaming_integrated_policy_and_planning_for_climate_resilient_sustainable_development',
#        'ecosystem_based_adaptation', 'urban_resilience',
#        'climate_resilient_integrated_water_resource_and_coastal_management']

# climate_forest_of_climate_change_mitigation = ['carbon_conservation', 'carbon_sequestration',
#        'climate_financing_for_redd_redd']

# climate_strategies_and_policy_programme = []

# ecosystems_and_biodiversity_programme = ['ecosystem_management_and_restoration', 'mainstreaming_biodiversity',
#        'strengthening_conservation_areas']

# energy_program_of_climate_change_mitigation = ['energy_efficiency',
#        'renewable_energy', 'energy_access', 'miscellaneous_energy_program']

# oceans_and_water_programme = ['small_island_dev_states_program_sids',
#        'climate_resilient_access_to_water_supply_sanitation',
#        'climate_resilient_integrated_water_resource_coastal_area_management',
#        'cross_cutting_governance_and_learning',
#        'protection_of_transboundary_surface_groundwater_resources_in_a_changing_climate',
#        'sustainable_management_of_oceans_in_a_changing_climate']

# environmental_governance_programme = ['mainstreaming_green_economy',
#        'gcp_green_commodities_programme']

# sub_categories = [chemicals_and_waste_programme, climate_change_adaptation_programme, 
#                   climate_forest_of_climate_change_mitigation, 
#                   climate_strategies_and_policy_programme ,ecosystems_and_biodiversity_programme, 
#                   energy_program_of_climate_change_mitigation, 
#                   oceans_and_water_programme, environmental_governance_programme]

# d_3 = dict(zip(parents, sub_categories)) 


'''import dict'''
with open(os.path.abspath(os.path.join('..', 'data/processed/'))+"/technical_team_dict.pkl", 'rb') as handle:
    technical_team_dict = pickle.load(handle)

In [123]:
'''hot_topics'''
hot_topics = ['cov_19', 'crisis_setting', 'digital_transformation',
       'food_and_agricultural_commodities_strategy', 'green_recovery',
       'health', 'human_rights', 'leaving_no_one_behind',
       'multi_stakeholder_collaboration', 'nature_based_solution', 
       'plastic', 'poverty_reduction', 'public_private_partnership', 'sids',
       'south_south_cooperation', 'structural_system_transformation']

sub_categories = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
d_4 = dict(zip(hot_topics, sub_categories)) 

hot_topics = []
for key, values in d_4.items():
    hot_topics.append(key)
hot_topics_parent = {'hot_topics': hot_topics}

In [124]:
'''conventions_protocols'''

conventions_protocols = ['cartagena_protocol_on_biosafety_lmos_gmos',
 'convention_on_biological_diversity_cbd',
 'international_maritime_organization_conventions_ime',
 'lme_conventions',
 'minamata_convention_on_mercury',
 'montreal_protocols',
 'multi_national_plans',
 'nagoya_protocol_abs',
 'national_action_plan',
 'national_adaptation_plan_nap',
 'national_biodiversity_strategies_and_action_plans_nbsaps',
 'national_communications',
 'national_determined_contributions_ndcs',
 'national_development_policies',
 'other_global_conventions',
 'regional_seas_conventions_and_action_plans',
 'stockholm_convention_pops',
 'strategic_action_programme',
 'the_convention_on_international_trade_in_endangered_species_of_wild_fauna_and_flora_cites',
 'united_nations_convention_to_combat_desertification_unccd',
 'united_nations_framework_convention_on_climate_change_unfccc']

sub_categories = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
d_5 = dict(zip(conventions_protocols, sub_categories)) 

conventions_protocols_parent = {'conventions_protocols': conventions_protocols}

In [125]:
'''gender_equality'''

gender_equality = ['awareness_raising_on_gender', 'gender_based_violence',
       'gender_responsive_policies', 'land_rights_for_women',
       'livelihoods_for_women', 'women_decision_making',
       'women_farmers', 'women_park_rangers',
       'women_s_access_to_and_control_over_resources',
       'women_s_cooperatives_and_groups']

sub_categories = [[], [], [], [], [], [], [], [], [], []]
d_6 = dict(zip(gender_equality, sub_categories))

gender_equality_parent = {'gender_equality': gender_equality}

In [126]:
'''pathways'''

pathways = [
 'people_pathway',
 'sci_tech_pathway',
 'systems_pathway']

sub_categories = [[], [], []]
d_7 = dict(zip(pathways, sub_categories))

pathways_parent = {'pathways': pathways}

In [127]:
'''private_sector'''

private_sector = ['capital_providers',
       'entrepreneurs_manufacturing_firms_investors',
       'financial_intermediaries_and_market_facilitators',
       'individuals_entrepreneurs', 'iron_and_steel', 'large_corporations',
       'manufacturers_building_professionals',  'non_grant_pilot',
       'poultry_fisheries_dairy_horticulture_khadi_homespun_cloth_and_silk_weaving_bamboo_and_commercial_cooking',
       'retailers_manufacturers', 'small_and_medium_sized_enterprises',
       'sugarcane_mills']

sub_categories = [[], [], [], [], [], [],[], [], [],[], [], []]
d_8 = dict(zip(private_sector, sub_categories))

private_sector_parent = {'private_sector': private_sector}

In [128]:
'''social_inclusion'''

social_inclusion = ['artisanal_miners', 'disabled',
       'elderly', 'indigenous_peoples', 'local_community_csos',
       'private_sector', 'smallholder_farmers', 'waste_picker', 'women',
       'youth_children']

sub_categories = [[], [], [], [], [], [],[], [], [],[]]
d_9 = dict(zip(social_inclusion, sub_categories))

social_inclusion_parent = {'social_inclusion': social_inclusion}

In [129]:
'''targeted_risk'''

targeted_risk = ['hazard_control_mitigation',
 'improve_resilience',
 'reduce_exposure']

sub_categories = [[], [], []]
d_10 = dict(zip(targeted_risk, sub_categories))

targeted_risk_parent = {'targeted_risk': targeted_risk}

In [130]:
'''transformed_sectors'''

transformed_sectors = ['_asgm', '_coal', '_metals_and_mining',
       '_oil_gas_and_fuels', '_shipping', 'agriculture', 'aquaculture',
       'electronic', 'energy', 'fisheries', 'forestry_and_other_land_use',
       'health_care', 'high_tech', 'livestock', 'materials_and_manufacturing',
        'other_sectors', 'retail', 'tourism', 'transportation']

sub_categories = [[], [], [],[], [], [],[], [], [],[], [], [],[], [], [],[], [], [],[]]
d_11 = dict(zip(transformed_sectors, sub_categories))

transformed_sectors_parent = {'transformed_sectors': transformed_sectors}

In [131]:
'''undp_roles'''

undp_roles = ['capacity_development_technical_assistance',
       'convening_partnerships_knowledge_sharing',
       'data_collection_and_analysis', 'direct_support_service_delivery',
       'innovative_approaches', 'institutional_mechanism_and_system_building',
         'normative_support', 'optimising_financing', 'policy_advice',
       'risk_analysis', 'support_functions', 'thought_leadership']

sub_categories = [[], [], [],[], [], [],[], [], [],[], [], []]
d_12 = dict(zip(undp_roles, sub_categories))

undp_roles_parent = {'undp_roles': undp_roles}

In [132]:
full_dict = {}
full_dict.update(landscape_dict)
#full_dict = landscape_dict_copy
full_dict.update(strategy_dict)
full_dict.update(technical_team_dict)
full_dict.update(d_4)
full_dict.update(d_5)
full_dict.update(d_6)
full_dict.update(d_7)
full_dict.update(d_8)
full_dict.update(d_9)
full_dict.update(d_10)
full_dict.update(d_11)
full_dict.update(d_12)

In [133]:
with open(os.path.abspath(os.path.join('..', 'data/processed/'))+"/category_dict.pkl", 'wb') as handle:
    pickle.dump(full_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Generate Lists

In [134]:
technical_team = []
technical_team_sub = []
for key, values in technical_team_dict.items():
    technical_team.append(key)
    for sub in values:
        if sub not in technical_team_sub:
            technical_team_sub.append(sub)
technical_team_parent = {'technical_team': technical_team}

landscapes = []
landscapes_sub = []
for key, values in landscape_dict.items():
    landscapes.append(key)
    for sub in values:
        if sub not in landscapes_sub:
            landscapes_sub.append(sub)
landscapes_parent = {'landscapes': landscapes}

strategies = []
strategies_sub = []
for key, values in strategy_dict.items():
    strategies.append(key)
    for sub in values:
        if sub not in strategies_sub:
            strategies_sub.append(sub)
strategies_parent = {'strategies': strategies}


In [135]:
parent = {}
parent.update(landscapes_parent)
parent.update(strategies_parent)
parent.update(technical_team_parent)
parent.update(hot_topics_parent)
parent.update(conventions_protocols_parent)
parent.update(gender_equality_parent)
parent.update(pathways_parent)
parent.update(private_sector_parent)
parent.update(social_inclusion_parent)
parent.update(targeted_risk_parent)
parent.update(transformed_sectors_parent)
parent.update(undp_roles_parent)

with open(os.path.abspath(os.path.join('..', 'data/processed/'))+"/parent_dict.pkl", 'wb') as handle:
    pickle.dump(parent, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [136]:
header = []
for key, values in parent.items():
    header.append(key)
header

['landscapes',
 'strategies',
 'technical_team',
 'hot_topics',
 'conventions_protocols',
 'gender_equality',
 'pathways',
 'private_sector',
 'social_inclusion',
 'targeted_risk',
 'transformed_sectors',
 'undp_roles']

In [137]:
full_dict

{'forests': ['dryland_forests',
  'protected_areas_network',
  'intact_forests',
  'tropical_forests',
  'montane_forests',
  'temperate_forests',
  'marine_and_coastal_protected_areas',
  'specially_protected_areas_spas',
  'productive_landscapes_seascapes',
  'estuaries',
  'grazing_lands',
  'rivers_and_river_basins',
  'coasts',
  'coral_reefs',
  'mangroves',
  'seas',
  'swamps',
  'boreal_forests_taiga_forests',
  'important_bird_and_biodiversity_areas_ibas',
  'key_biodiversity_areas_kbas',
  'terrestrial_protected_areas',
  'rural_areas',
  'urban_areas'],
 'tundra': ['alpine_tundra'],
 'conserve_areas': ['key_biodiversity_areas_kbas',
  'terrestrial_protected_areas',
  'important_bird_and_biodiversity_areas_ibas',
  '_important_bird_and_biodiversity_areas_ibas',
  'marine_and_coastal_protected_areas',
  'specially_protected_areas_spas',
  'productive_landscapes_seascapes',
  'protected_areas_network',
  'locally_managed_marine_areas',
  'transboundary_conservation_areas',
  '

In [144]:
print("wanna check sub-categories as well?")
# if all:
#     sub_categories
for key, value in parent.items():
    if key == "landscapes":
        a = value
sub_categories = []
for key, value in full_dict.items():
    for i in a:
        if i == key:
            sub_categories
            sub_categories.append(value)
sub_dict = dict(zip(a, sub_categories))
sub_categories

[['dryland_forests',
  'protected_areas_network',
  'intact_forests',
  'tropical_forests',
  'montane_forests',
  'temperate_forests',
  'marine_and_coastal_protected_areas',
  'specially_protected_areas_spas',
  'productive_landscapes_seascapes',
  'estuaries',
  'grazing_lands',
  'rivers_and_river_basins',
  'coasts',
  'coral_reefs',
  'mangroves',
  'seas',
  'swamps',
  'boreal_forests_taiga_forests',
  'important_bird_and_biodiversity_areas_ibas',
  'key_biodiversity_areas_kbas',
  'terrestrial_protected_areas',
  'rural_areas',
  'urban_areas'],
 ['alpine_tundra'],
 ['key_biodiversity_areas_kbas',
  'terrestrial_protected_areas',
  'important_bird_and_biodiversity_areas_ibas',
  '_important_bird_and_biodiversity_areas_ibas',
  'marine_and_coastal_protected_areas',
  'specially_protected_areas_spas',
  'productive_landscapes_seascapes',
  'protected_areas_network',
  'locally_managed_marine_areas',
  'transboundary_conservation_areas',
  'indigenous_and_communities_conserved_ar

In [96]:
full_dict

{'forests': ['dryland_forests',
  'protected_areas_network',
  'intact_forests',
  'tropical_forests',
  'montane_forests',
  'temperate_forests',
  'marine_and_coastal_protected_areas',
  'specially_protected_areas_spas',
  'productive_landscapes_seascapes',
  'estuaries',
  'grazing_lands',
  'rivers_and_river_basins',
  'coasts',
  'coral_reefs',
  'mangroves',
  'seas',
  'swamps',
  'boreal_forests_taiga_forests',
  'important_bird_and_biodiversity_areas_ibas',
  'key_biodiversity_areas_kbas',
  'terrestrial_protected_areas',
  'rural_areas',
  'urban_areas'],
 'tundra': ['alpine_tundra'],
 'conserve_areas': ['key_biodiversity_areas_kbas',
  'terrestrial_protected_areas',
  'important_bird_and_biodiversity_areas_ibas',
  '_important_bird_and_biodiversity_areas_ibas',
  'marine_and_coastal_protected_areas',
  'specially_protected_areas_spas',
  'productive_landscapes_seascapes',
  'protected_areas_network',
  'locally_managed_marine_areas',
  'transboundary_conservation_areas',
  '

In [102]:
parent

{'landscapes': ['forests',
  'tundra',
  'conserve_areas',
  'freshwaters',
  'grasslands',
  'deserts',
  'marine',
  'wetlands',
  'human_altered_areas'],
 'strategies': ['capacity_building',
  'enabling',
  'finance_economy',
  'food_and_agricultural_commodities',
  'governance',
  'law_regulation',
  'management_operation',
  'mitigation_adaptation',
  'monitor_inventory',
  'technology_innovation'],
 'technical_team': ['ecosystems_and_biodiversity_programme',
  'chemicals_and_waste_programme',
  'energy_program_of_climate_change_mitigation',
  'oceans_and_water_programme',
  'climate_forest_of_climate_change_mitigation',
  'climate_change_adaptation_programme',
  'environmental_governance_programme',
  'climate_strategies_and_policy_programme'],
 'hot_topics': ['cov_19',
  'crisis_setting',
  'digital_transformation',
  'food_and_agricultural_commodities_strategy',
  'green_recovery',
  'health',
  'human_rights',
  'leaving_no_one_behind',
  'multi_stakeholder_collaboration',
  '